# Importing libraries

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split

# Importing the dataset

In [4]:
data=pd.read_csv('gas_turbines.csv')
data.head() #retrieving the first 5 records

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311


# EDA

In [5]:
print('The dimension of the dataset is {}'.format(data.shape)) #no. of rows and columns in the dataset

The dimension of the dataset is (15039, 11)


## Checking for null values:

In [6]:
data.isnull().sum() #there are no null values

AT      0
AP      0
AH      0
AFDP    0
GTEP    0
TIT     0
TAT     0
TEY     0
CDP     0
CO      0
NOX     0
dtype: int64

In [7]:
data.info() #checking the data types of each attribute

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15039 entries, 0 to 15038
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      15039 non-null  float64
 1   AP      15039 non-null  float64
 2   AH      15039 non-null  float64
 3   AFDP    15039 non-null  float64
 4   GTEP    15039 non-null  float64
 5   TIT     15039 non-null  float64
 6   TAT     15039 non-null  float64
 7   TEY     15039 non-null  float64
 8   CDP     15039 non-null  float64
 9   CO      15039 non-null  float64
 10  NOX     15039 non-null  float64
dtypes: float64(11)
memory usage: 1.3 MB


In [8]:
data.describe() #all the statistics of te features

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
count,15039.000000,15039.00000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000
mean,17.764381,1013.19924,79.124174,4.200294,25.419061,1083.798770,545.396183,134.188464,12.102353,1.972499,68.190934
std,7.574323,6.41076,13.793439,0.760197,4.173916,16.527806,7.866803,15.829717,1.103196,2.222206,10.470586
min,0.522300,985.85000,30.344000,2.087400,17.878000,1000.800000,512.450000,100.170000,9.904400,0.000388,27.765000
25%,11.408000,1008.90000,69.750000,3.723900,23.294000,1079.600000,542.170000,127.985000,11.622000,0.858055,61.303500
50%,18.186000,1012.80000,82.266000,4.186200,25.082000,1088.700000,549.890000,133.780000,12.025000,1.390200,66.601000
75%,23.862500,1016.90000,90.043500,4.550900,27.184000,1096.000000,550.060000,140.895000,12.578000,2.160400,73.935500
max,34.929000,1034.20000,100.200000,7.610600,37.402000,1100.800000,550.610000,174.610000,15.081000,44.103000,119.890000


###### I am using dtale library to obtain all the visualisation plots and statistics of the dataset

In [9]:
import dtale
dtale.show(data)

###### Additionally using sweetviz library to obtain the statistics.

In [10]:
import sweetviz as sv
report=sv.analyze(data)
report.show_html('sweet_report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report sweet_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [11]:
X=data.drop('TEY',axis=1) #Getting all the independent features
y=data['TEY'] #Dependent feature

In [12]:
data.columns #Names of all the columns

Index(['AT', 'AP', 'AH', 'AFDP', 'GTEP', 'TIT', 'TAT', 'TEY', 'CDP', 'CO',
       'NOX'],
      dtype='object')

# Feature Selection

##### The Artificial neural network assumes that the input features are independent. There is a possibility of  multicollinearity between the variables. Thus I will set a threshold and whichever has the correlation more than the threshold will be dropped.

In [13]:
plt.figure(figsize=(12,10))
corr=X.corr()
sns.heatmap(corr,annot=True,cmap=plt.cm.CMRmap_r)
plt.show()

<ipython-input-13-d03a6c6b7107>:4: UserWarning:

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.



In [14]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset,threshold):
    col_corr=set()
    corr_matrix=dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if(abs(corr_matrix.iloc[i,j]>threshold)):
                colname=corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [15]:
corr_features = correlation(X, 0.7) #threshold is 0.7
len(set(corr_features))

3

In [16]:
corr_features

{'CDP', 'GTEP', 'TIT'}

In [17]:
X.drop(corr_features,axis=1) #dropping the features which are highly correlated.

,AT,AP,AH,AFDP,TAT,CO,NOX
0,6.8594,1007.9,96.799,3.5000,550.00,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,550.00,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,549.87,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,549.99,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,549.98,3.2484,82.311
...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,546.21,4.5186,79.559
15035,7.8879,1005.9,99.093,3.5059,543.22,4.8470,79.917
15036,7.2647,1006.3,99.496,3.4770,537.32,7.9632,90.912
15037,7.0060,1006.8,99.008,3.4486,541.24,6.2494,93.227


# Feature Scaling:

##### ### Performing Standardisation using sklearn MinmaxSclaer and fitting it to train and test dataset.Since the high magnitude features are dominant over the low magnitude ones, we perform standarisation of the features using the MinmaxSclaer. This will transform the values between 0 and 1.

In [18]:
from sklearn.preprocessing import MinMaxScaler
Scaler=MinMaxScaler()
X=Scaler.fit_transform(X)
y=y.to_numpy()
print(X)
print(y)

[[0.18418215 0.45604964 0.95131413 ... 0.1353398  0.07152212 0.59654817]
 [0.18201978 0.4663909  0.95588067 ... 0.13398756 0.07337235 0.59713433]
 [0.18529531 0.47466391 0.93900309 ... 0.13456709 0.07257648 0.59379104]
 ...
 [0.19596183 0.4229576  0.98992213 ... 0.11177221 0.18055195 0.68544912]
 [0.18844295 0.43329886 0.98293633 ... 0.12143106 0.14169257 0.71057802]
 [0.18617304 0.44157187 0.96182146 ... 0.1310899  0.11294597 0.70266486]]
[114.7  114.72 114.71 ... 110.19 110.74 111.58]


In [19]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=56) #splitting the train and test dataset with
#test size as 0.3 and random state as 56 for reproducibility.

# Building Model:

##### Building an Artificial neural network and performing hyperparameter tuning on the no. of layers and the neurons. Activation function is linear since this is a regression problem.

In [18]:
#defining the function build_model
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units',
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [22]:
tuner=RandomSearch(
   build_model,
   objective='val_mean_absolute_error',
   max_trials=5,
   executions_per_trial=3,
   directory='project',
   project_name='Gas Turbine_1')

In [23]:
tuner.search_space_summary()

In [24]:
tuner.search(x_train, y_train,
             epochs=5,
             validation_data=(x_test, y_test))

Epoch 1/5
329/329 [==============================] - ETA: 0s - loss: 136.8914 - mean_absolute_error: 136.891 - ETA: 1s - loss: 134.8884 - mean_absolute_error: 134.888 - ETA: 1s - loss: 134.5708 - mean_absolute_error: 134.570 - ETA: 1s - loss: 133.8934 - mean_absolute_error: 133.893 - ETA: 0s - loss: 133.3731 - mean_absolute_error: 133.373 - ETA: 0s - loss: 130.2379 - mean_absolute_error: 130.237 - ETA: 0s - loss: 121.5373 - mean_absolute_error: 121.537 - ETA: 0s - loss: 105.9296 - mean_absolute_error: 105.929 - ETA: 0s - loss: 90.6908 - mean_absolute_error: 90.6908  - ETA: 0s - loss: 81.7945 - mean_absolute_error: 81.794 - ETA: 0s - loss: 74.0394 - mean_absolute_error: 74.039 - ETA: 0s - loss: 67.6556 - mean_absolute_error: 67.655 - ETA: 0s - loss: 62.2949 - mean_absolute_error: 62.294 - ETA: 0s - loss: 57.7620 - mean_absolute_error: 57.762 - ETA: 0s - loss: 54.0700 - mean_absolute_error: 54.070 - ETA: 0s - loss: 50.6628 - mean_absolute_error: 50.662 - ETA: 0s - loss: 47.6866 - mean_ab

Epoch 1/5
329/329 [==============================] - ETA: 0s - loss: 130.2146 - mean_absolute_error: 130.214 - ETA: 2s - loss: 127.0321 - mean_absolute_error: 127.032 - ETA: 2s - loss: 96.0878 - mean_absolute_error: 96.0878  - ETA: 2s - loss: 82.2875 - mean_absolute_error: 82.287 - ETA: 2s - loss: 70.0736 - mean_absolute_error: 70.073 - ETA: 2s - loss: 62.1835 - mean_absolute_error: 62.183 - ETA: 2s - loss: 55.6108 - mean_absolute_error: 55.610 - ETA: 3s - loss: 53.8579 - mean_absolute_error: 53.857 - ETA: 3s - loss: 49.7128 - mean_absolute_error: 49.712 - ETA: 3s - loss: 45.9606 - mean_absolute_error: 45.960 - ETA: 3s - loss: 42.9704 - mean_absolute_error: 42.970 - ETA: 3s - loss: 39.6900 - mean_absolute_error: 39.690 - ETA: 3s - loss: 36.8100 - mean_absolute_error: 36.810 - ETA: 3s - loss: 34.6875 - mean_absolute_error: 34.687 - ETA: 3s - loss: 32.8724 - mean_absolute_error: 32.872 - ETA: 3s - loss: 31.2362 - mean_absolute_error: 31.236 - ETA: 3s - loss: 29.7639 - mean_absolute_error

Epoch 1/5
325/329 [============================>.] - ETA: 0s - loss: 132.3031 - mean_absolute_error: 132.303 - ETA: 6s - loss: 6283.9937 - mean_absolute_error: 6283.993 - ETA: 7s - loss: 3811.7876 - mean_absolute_error: 3811.787 - ETA: 7s - loss: 2760.5740 - mean_absolute_error: 2760.574 - ETA: 8s - loss: 2176.0557 - mean_absolute_error: 2176.055 - ETA: 7s - loss: 1804.4432 - mean_absolute_error: 1804.443 - ETA: 7s - loss: 1442.4226 - mean_absolute_error: 1442.422 - ETA: 7s - loss: 1266.8519 - mean_absolute_error: 1266.851 - ETA: 7s - loss: 1075.3438 - mean_absolute_error: 1075.343 - ETA: 7s - loss: 976.6687 - mean_absolute_error: 976.6687  - ETA: 7s - loss: 896.8032 - mean_absolute_error: 896.803 - ETA: 7s - loss: 827.5916 - mean_absolute_error: 827.591 - ETA: 7s - loss: 767.7972 - mean_absolute_error: 767.797 - ETA: 7s - loss: 716.7710 - mean_absolute_error: 716.771 - ETA: 7s - loss: 652.4045 - mean_absolute_error: 652.404 - ETA: 7s - loss: 598.5375 - mean_absolute_error: 598.537 - E

Epoch 1/5
329/329 [==============================] - ETA: 0s - loss: 136.4042 - mean_absolute_error: 136.404 - ETA: 4s - loss: 112.7442 - mean_absolute_error: 112.744 - ETA: 6s - loss: 120.2170 - mean_absolute_error: 120.217 - ETA: 6s - loss: 207.6847 - mean_absolute_error: 207.684 - ETA: 6s - loss: 178.7536 - mean_absolute_error: 178.753 - ETA: 6s - loss: 169.3395 - mean_absolute_error: 169.339 - ETA: 6s - loss: 165.1919 - mean_absolute_error: 165.191 - ETA: 6s - loss: 160.6402 - mean_absolute_error: 160.640 - ETA: 6s - loss: 158.0952 - mean_absolute_error: 158.095 - ETA: 6s - loss: 155.0396 - mean_absolute_error: 155.039 - ETA: 6s - loss: 150.1572 - mean_absolute_error: 150.157 - ETA: 6s - loss: 144.9567 - mean_absolute_error: 144.956 - ETA: 6s - loss: 139.5115 - mean_absolute_error: 139.511 - ETA: 6s - loss: 131.6935 - mean_absolute_error: 131.693 - ETA: 6s - loss: 128.1779 - mean_absolute_error: 128.177 - ETA: 6s - loss: 126.2003 - mean_absolute_error: 126.200 - ETA: 6s - loss: 121

Epoch 1/5
329/329 [==============================] - ETA: 0s - loss: 135.4925 - mean_absolute_error: 135.492 - ETA: 6s - loss: 133.5614 - mean_absolute_error: 133.561 - ETA: 7s - loss: 133.3994 - mean_absolute_error: 133.399 - ETA: 8s - loss: 133.5194 - mean_absolute_error: 133.519 - ETA: 8s - loss: 133.4424 - mean_absolute_error: 133.442 - ETA: 9s - loss: 133.6933 - mean_absolute_error: 133.693 - ETA: 8s - loss: 133.7853 - mean_absolute_error: 133.785 - ETA: 8s - loss: 134.1545 - mean_absolute_error: 134.154 - ETA: 8s - loss: 133.9695 - mean_absolute_error: 133.969 - ETA: 7s - loss: 133.6869 - mean_absolute_error: 133.686 - ETA: 7s - loss: 132.9778 - mean_absolute_error: 132.977 - ETA: 7s - loss: 132.0332 - mean_absolute_error: 132.033 - ETA: 7s - loss: 130.1393 - mean_absolute_error: 130.139 - ETA: 7s - loss: 126.7833 - mean_absolute_error: 126.783 - ETA: 7s - loss: 120.3340 - mean_absolute_error: 120.334 - ETA: 6s - loss: 113.4964 - mean_absolute_error: 113.496 - ETA: 6s - loss: 107

INFO:tensorflow:Oracle triggered exit


2021-09-03 16:58:54,094 - INFO     - Oracle triggered exit


In [25]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal no. of layers is {best_hps.get('num_layers')}The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal no. of layers is 15The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer
is 0.0001.



In [29]:
#Building the model with the best hyperparameters
hypermodel = tuner.hypermodel.build(best_hps)

In [32]:
hypermodel.fit(x_train, y_train, epochs=5, validation_split=0.2)

Epoch 1/5
264/264 [==============================] - ETA: 0s - loss: 1.1266 - mean_absolute_error: 1.126 - ETA: 3s - loss: 1.3528 - mean_absolute_error: 1.352 - ETA: 4s - loss: 1.2503 - mean_absolute_error: 1.250 - ETA: 4s - loss: 1.2757 - mean_absolute_error: 1.275 - ETA: 4s - loss: 1.2899 - mean_absolute_error: 1.289 - ETA: 4s - loss: 1.2790 - mean_absolute_error: 1.279 - ETA: 4s - loss: 1.2829 - mean_absolute_error: 1.282 - ETA: 4s - loss: 1.2745 - mean_absolute_error: 1.274 - ETA: 4s - loss: 1.2718 - mean_absolute_error: 1.271 - ETA: 4s - loss: 1.2489 - mean_absolute_error: 1.248 - ETA: 4s - loss: 1.2302 - mean_absolute_error: 1.230 - ETA: 4s - loss: 1.2074 - mean_absolute_error: 1.207 - ETA: 4s - loss: 1.1963 - mean_absolute_error: 1.196 - ETA: 3s - loss: 1.2096 - mean_absolute_error: 1.209 - ETA: 3s - loss: 1.2643 - mean_absolute_error: 1.264 - ETA: 3s - loss: 1.2928 - mean_absolute_error: 1.292 - ETA: 3s - loss: 1.3075 - mean_absolute_error: 1.307 - ETA: 3s - loss: 1.3458 - mean

In [39]:
eval_result = hypermodel.evaluate(x_test, y_test)

141/141 [==============================] - ETA: 0s - loss: 3.0442 - mean_absolute_error: 3.044 - ETA: 0s - loss: 2.5530 - mean_absolute_error: 2.553 - ETA: 0s - loss: 2.4566 - mean_absolute_error: 2.456 - ETA: 0s - loss: 2.4609 - mean_absolute_error: 2.460 - ETA: 0s - loss: 2.4764 - mean_absolute_error: 2.476 - ETA: 0s - loss: 2.4648 - mean_absolute_error: 2.464 - ETA: 0s - loss: 2.4599 - mean_absolute_error: 2.459 - ETA: 0s - loss: 2.4630 - mean_absolute_error: 2.463 - ETA: 0s - loss: 2.4677 - mean_absolute_error: 2.467 - ETA: 0s - loss: 2.4562 - mean_absolute_error: 2.456 - ETA: 0s - loss: 2.4591 - mean_absolute_error: 2.459 - ETA: 0s - loss: 2.4626 - mean_absolute_error: 2.462 - 1s 4ms/step - loss: 2.4661 - mean_absolute_error: 2.4661
test accuracy 97.53392505645752


In [40]:
print("test accuracy", (100-eval_result[0]))

test accuracy 97.53392505645752


##### We have got an accuracy of 97.533 which is great.